In [1]:
param_market = 'US'

In [ ]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./item.xlsx',
                       header=0,
                       dtype=str).fillna('')

dict_item = {}
list_job = []
for i in range(len(input_)):
    if input_.loc[i, 'Vehicle_Page'] != '':
        vehicle_page = int(input_.loc[i, 'Vehicle_Page'])
        dict_item[input_.loc[i, 'Item_Number']] = {'vehicle_page': vehicle_page,
                                                   'crawler': 0,
                                                   'dict_vehicle': {},
                                                   'vehicle_1': '',
                                                   'vehicle_2': ''}
        for j in range(vehicle_page):
            list_job.append((i, j))

print('总数量：' + str(len(list_job)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(list_job)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(100):
            try:
                i, j = list_job[a]

                data = '{"scopedContext":{"catalogDetails":{"itemId":"' + input_.loc[i, 'Item_Number'] + '","categoryId":"' + input_.loc[i, 'Category_Id'] + '","marketplaceId":"' + input_.loc[i, 'Market_Id'] + '"}}}'

                ua = UA.get_User_Agent_Requests()
                ua['Content-Type'] = 'application/json'

                if param_market == 'US':
                    url = 'https://www.ebay.com/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset=' + str(j*20) + '&module=COMPATIBILITY_TABLE&_ul=US&_stpos=91710&orig_cvip=true'
                elif param_market == 'DE':
                    url = 'https://www.ebay.de/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset=' + str(j*20) + '&module=COMPATIBILITY_TABLE&_ul=DE&_stpos=10115&orig_cvip=true'
                # elif param_market == 'AU':
                #     pass
                elif param_market == 'UK':
                    url = 'https://www.ebay.co.uk/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset=' + str(j*20) + '&module=COMPATIBILITY_TABLE&_ul=GB&_stpos=BS81QU&orig_cvip=true'

                resp = requests.post(url,
                                     data=data,
                                     headers=ua,
                                     proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                json_data = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                list_header = json_data['modules']['COMPATIBILITY_TABLE']['paginatedTable']['header']['cells']
                
                # = = = = = = = = = = = = = = =
                
                if param_market in ['US', 'AU', 'UK']:
                    name_make = 'Make'
                    name_model = 'Model'
                    name_engine = 'Engine'
                    name_year = 'Year'
                elif param_market == 'DE':
                    name_make = 'Marke'
                    name_model = 'Modell'
                    name_engine = 'Motor'
                    name_year = 'Baujahr'

                for k in range(len(list_header)):
                    if name_make in list_header[k]['textSpans'][0]['text']:
                        col_make = k
                    elif name_model in list_header[k]['textSpans'][0]['text']:
                        col_model = k
                    elif name_engine in list_header[k]['textSpans'][0]['text']:
                        col_engine = k
                    elif name_year in list_header[k]['textSpans'][0]['text']:
                        col_year = k
                        
                # = = = = = = = = = = = = = = =
                
                list_row = json_data['modules']['COMPATIBILITY_TABLE']['paginatedTable']['rows']

                # = = = = = = = = = = = = = = =
                
                for row in list_row:
                    make = row['cells'][col_make]['textSpans'][0]['text'].strip()
                    model = row['cells'][col_model]['textSpans'][0]['text'].strip()
                    engine = row['cells'][col_engine]['textSpans'][0]['text'].strip()
                    if param_market in ['US', 'AU', 'UK']:
                        list_year = [int(row['cells'][col_year]['textSpans'][0]['text'].strip())]
                    elif param_market == 'DE':
                        list_year = [int(year.split('/').strip()) for year in row['cells'][col_year]['textSpans'][0]['text'].split('-')]

                    if make not in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle']:
                        dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make] = {model: {engine: list_year}}
                    else:
                        if model not in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make]:
                            dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model] = {engine: list_year}
                        else:
                            if engine not in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model]:
                                dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine] = list_year
                            else:
                                dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine] += list_year

                    dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine] = list(set(dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine]))
                    dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine].sort()

                dict_item[input_.loc[i, 'Item_Number']]['crawler'] += 1

                # = = = = = = = = = = = = = = =
                
                if dict_item[input_.loc[i, 'Item_Number']]['vehicle_page'] == dict_item[input_.loc[i, 'Item_Number']]['crawler']:
                    list_vehicle = []
                    for make in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle']:
                        for model in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make]:
                            list_year = []
                            for engine in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model]:
                                list_year += dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine]
                            list_year = list(set(list_year))
                            list_year.sort()

                            year_min = list_year[0]
                            year_max = list_year[-1]
                            if year_min == year_max:
                                list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
                            else:
                                list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))

                    list_vehicle.sort()

                    dict_item[input_.loc[i, 'Item_Number']]['vehicle_1'] = '\n'.join(list_vehicle)
                    
                    # = = = = = = = = = = = = = = =
                    
                    list_vehicle = []
                    for make in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle']:
                        for model in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make]:
                            for engine in dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model]:
                                year_min = dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine][0]
                                year_max = dict_item[input_.loc[i, 'Item_Number']]['dict_vehicle'][make][model][engine][-1]
                                if year_min == year_max:
                                    if engine == '':
                                        list_vehicle.append(make + ' ' + model + ' ' + str(year_min))
                                    else:
                                        list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + ' ' + engine)
                                else:
                                    if engine == '':
                                        list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max))
                                    else:
                                        list_vehicle.append(make + ' ' + model + ' ' + str(year_min) + '-' + str(year_max) + ' ' + engine)

                    list_vehicle.sort()

                    dict_item[input_.loc[i, 'Item_Number']]['vehicle_2'] = '\n'.join(list_vehicle)
                    
                    # = = = = = = = = = = = = = = =
                    
                    df_temp = pd.DataFrame([{'Item_Number': input_.loc[i, 'Item_Number'],
                                             'Vehicle_1': dict_item[input_.loc[i, 'Item_Number']]['vehicle_1'],
                                             'Vehicle_2': dict_item[input_.loc[i, 'Item_Number']]['vehicle_2']}])

                    output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'

                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'Item_Number': input_.loc[i, 'Item_Number'],
                                     'Vehicle_Page': input_.loc[i, 'Vehicle_Page'],
                                     'Category_Id': input_.loc[i, 'Category_Id'],
                                     'Market_Id': input_.loc[i, 'Market_Id']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[i, 'Item_Number'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(10):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['Item_Number'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./vehicle-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['Item_Number'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./vehicle_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：9078

[ok] - 386036291998
[尝试次数：1] - [剩余数量：9068] - [当前时间：17:40:58]

[ok] - 386036291998
[尝试次数：1] - [剩余数量：9067] - [当前时间：17:40:58]

[ok] - 384810665521
[尝试次数：1] - [剩余数量：9066] - [当前时间：17:40:58]

[ok] - 386036291998
[尝试次数：1] - [剩余数量：9065] - [当前时间：17:40:58]

[ok] - 386063642468
[尝试次数：1] - [剩余数量：9064] - [当前时间：17:40:58]

[ok] - 384810665521
[尝试次数：1] - [剩余数量：9063] - [当前时间：17:40:58]

[ok] - 386063642468
[尝试次数：1] - [剩余数量：9062] - [当前时间：17:40:58]

[ok] - 386036291998
[尝试次数：1] - [剩余数量：9061] - [当前时间：17:40:58]

[ok] - 386063642468
[尝试次数：1] - [剩余数量：9060] - [当前时间：17:40:59]

[ok] - 386063642468
[尝试次数：1] - [剩余数量：9059] - [当前时间：17:40:59]

[ok] - 386063642468
[尝试次数：1] - [剩余数量：9058] - [当前时间：17:40:59]

[ok] - 386063642468
[尝试次数：1] - [剩余数量：9057] - [当前时间：17:40:59]

[ok] - 386063642468
[尝试次数：1] - [剩余数量：9056] - [当前时间：17:40:59]

[ok] - 386063642468
[尝试次数：1] - [剩余数量：9055] - [当前时间：17:40:59]

[ok] - 386063642468
[尝试次数：1] - [剩余数量：9054] - [当前时间：17:41:00]

[ok] - 386036291998
[尝试次数：1] - [剩余数量：9053] - [当前时间：17:41:00]

[ok] - 384642925431
[尝试次数：1] - [剩余数量：8936] - [当前时间：17:41:28]

[ok] - 386009670529
[尝试次数：1] - [剩余数量：8935] - [当前时间：17:41:29]

[ok] - 386009670529
[尝试次数：1] - [剩余数量：8934] - [当前时间：17:41:29]

[ok] - 386009670529
[尝试次数：1] - [剩余数量：8933] - [当前时间：17:41:29]

[ok] - 386009670529
[尝试次数：1] - [剩余数量：8932] - [当前时间：17:41:29]

[ok] - 384642925431
[尝试次数：1] - [剩余数量：8931] - [当前时间：17:41:29]

[ok] - 386009670529
[尝试次数：1] - [剩余数量：8930] - [当前时间：17:41:29]

[ok] - 386009670529
[尝试次数：1] - [剩余数量：8929] - [当前时间：17:41:30]

[ok] - 386009670529
[尝试次数：1] - [剩余数量：8928] - [当前时间：17:41:30]

[ok] - 386009670529
[尝试次数：1] - [剩余数量：8927] - [当前时间：17:41:30]

[ok] - 386009670529
[尝试次数：1] - [剩余数量：8926] - [当前时间：17:41:31]

[ok] - 386009670529
[尝试次数：1] - [剩余数量：8925] - [当前时间：17:41:31]

[ok] - 386009670529
[尝试次数：1] - [剩余数量：8924] - [当前时间：17:41:31]

[ok] - 386009670529
[尝试次数：1] - [剩余数量：8923] - [当前时间：17:41:31]

[ok] - 386009670529
[尝试次数：1] - [剩余数量：8922] - [当前时间：17:41:31]

[ok] - 386009670529
[尝试次数：1] - [剩余数量：8921] - [当前时间：17:41:32]

[ok] - 3

[ok] - 386017370863
[尝试次数：1] - [剩余数量：8803] - [当前时间：17:42:06]

[ok] - 386017370863
[尝试次数：1] - [剩余数量：8802] - [当前时间：17:42:06]

[ok] - 386017370863
[尝试次数：1] - [剩余数量：8801] - [当前时间：17:42:06]

[ok] - 386017370863
[尝试次数：1] - [剩余数量：8800] - [当前时间：17:42:06]

[ok] - 386017370863
[尝试次数：1] - [剩余数量：8799] - [当前时间：17:42:07]

[ok] - 386017370863
[尝试次数：1] - [剩余数量：8798] - [当前时间：17:42:07]

[ok] - 386017370863
[尝试次数：1] - [剩余数量：8797] - [当前时间：17:42:07]

[ok] - 386017370863
[尝试次数：1] - [剩余数量：8796] - [当前时间：17:42:07]

[ok] - 386017370863
[尝试次数：1] - [剩余数量：8795] - [当前时间：17:42:08]

[ok] - 386017370863
[尝试次数：1] - [剩余数量：8794] - [当前时间：17:42:08]

[ok] - 386017370863
[尝试次数：1] - [剩余数量：8793] - [当前时间：17:42:08]

[ok] - 386017370863
[尝试次数：1] - [剩余数量：8792] - [当前时间：17:42:09]

[ok] - 386017370863
[尝试次数：1] - [剩余数量：8791] - [当前时间：17:42:09]

[ok] - 386017370863
[尝试次数：1] - [剩余数量：8790] - [当前时间：17:42:09]

[ok] - 386017370863
[尝试次数：1] - [剩余数量：8789] - [当前时间：17:42:09]

[ok] - 386017370863
[尝试次数：1] - [剩余数量：8788] - [当前时间：17:42:09]

[ok] - 3

[ok] - 386030532462
[尝试次数：1] - [剩余数量：8670] - [当前时间：17:42:49]

[ok] - 386030532462
[尝试次数：2] - [剩余数量：8669] - [当前时间：17:42:49]

[ok] - 386010002814
[尝试次数：1] - [剩余数量：8668] - [当前时间：17:42:50]

[ok] - 386030532462
[尝试次数：1] - [剩余数量：8667] - [当前时间：17:42:50]

[ok] - 386030532462
[尝试次数：1] - [剩余数量：8666] - [当前时间：17:42:50]

[ok] - 386030532462
[尝试次数：1] - [剩余数量：8665] - [当前时间：17:42:50]

[ok] - 386030532462
[尝试次数：2] - [剩余数量：8664] - [当前时间：17:42:50]

[ok] - 386030532462
[尝试次数：1] - [剩余数量：8663] - [当前时间：17:42:51]

[ok] - 386030532462
[尝试次数：1] - [剩余数量：8662] - [当前时间：17:42:51]

[ok] - 386030532462
[尝试次数：2] - [剩余数量：8661] - [当前时间：17:42:51]

[ok] - 386030532462
[尝试次数：2] - [剩余数量：8660] - [当前时间：17:42:51]

[ok] - 386030532462
[尝试次数：1] - [剩余数量：8659] - [当前时间：17:42:51]

[ok] - 386030532462
[尝试次数：1] - [剩余数量：8658] - [当前时间：17:42:52]

[ok] - 386030532462
[尝试次数：1] - [剩余数量：8657] - [当前时间：17:42:52]

[ok] - 386030532462
[尝试次数：1] - [剩余数量：8656] - [当前时间：17:42:52]

[ok] - 386030532462
[尝试次数：1] - [剩余数量：8655] - [当前时间：17:42:52]

[ok] - 3

[ok] - 386209713366
[尝试次数：1] - [剩余数量：8537] - [当前时间：17:43:36]

[ok] - 386209713366
[尝试次数：1] - [剩余数量：8536] - [当前时间：17:43:36]

[ok] - 386063776483
[尝试次数：2] - [剩余数量：8535] - [当前时间：17:43:37]

[ok] - 386209713366
[尝试次数：1] - [剩余数量：8534] - [当前时间：17:43:37]

[ok] - 386209713366
[尝试次数：1] - [剩余数量：8533] - [当前时间：17:43:37]

[ok] - 386209713366
[尝试次数：1] - [剩余数量：8532] - [当前时间：17:43:37]

[ok] - 386209713366
[尝试次数：1] - [剩余数量：8531] - [当前时间：17:43:38]

[ok] - 386209713366
[尝试次数：1] - [剩余数量：8530] - [当前时间：17:43:38]

[ok] - 386209713366
[尝试次数：1] - [剩余数量：8529] - [当前时间：17:43:38]

[ok] - 386209713366
[尝试次数：1] - [剩余数量：8528] - [当前时间：17:43:38]

[ok] - 386209713366
[尝试次数：1] - [剩余数量：8527] - [当前时间：17:43:38]

[ok] - 386209713366
[尝试次数：1] - [剩余数量：8526] - [当前时间：17:43:39]

[ok] - 386209713366
[尝试次数：1] - [剩余数量：8525] - [当前时间：17:43:39]

[ok] - 386209713366
[尝试次数：1] - [剩余数量：8524] - [当前时间：17:43:39]

[ok] - 386209713366
[尝试次数：1] - [剩余数量：8523] - [当前时间：17:43:39]

[ok] - 386209713366
[尝试次数：1] - [剩余数量：8522] - [当前时间：17:43:39]

[ok] - 3

[ok] - 386030742722
[尝试次数：1] - [剩余数量：8403] - [当前时间：17:44:11]

[ok] - 386030742722
[尝试次数：1] - [剩余数量：8402] - [当前时间：17:44:11]

[ok] - 386030742722
[尝试次数：1] - [剩余数量：8401] - [当前时间：17:44:11]

[ok] - 386030742722
[尝试次数：1] - [剩余数量：8400] - [当前时间：17:44:12]

[ok] - 386030742722
[尝试次数：1] - [剩余数量：8399] - [当前时间：17:44:12]

[ok] - 386030742722
[尝试次数：6] - [剩余数量：8398] - [当前时间：17:44:12]

[ok] - 386030742722
[尝试次数：1] - [剩余数量：8397] - [当前时间：17:44:12]

[ok] - 386030742722
[尝试次数：1] - [剩余数量：8396] - [当前时间：17:44:13]

[ok] - 386030742722
[尝试次数：1] - [剩余数量：8395] - [当前时间：17:44:13]

[ok] - 386030742722
[尝试次数：1] - [剩余数量：8394] - [当前时间：17:44:13]

[ok] - 386030742722
[尝试次数：2] - [剩余数量：8393] - [当前时间：17:44:13]

[ok] - 386030742722
[尝试次数：2] - [剩余数量：8392] - [当前时间：17:44:13]

[ok] - 386030742722
[尝试次数：2] - [剩余数量：8391] - [当前时间：17:44:14]

[ok] - 386030742722
[尝试次数：1] - [剩余数量：8390] - [当前时间：17:44:14]

[ok] - 386030742722
[尝试次数：1] - [剩余数量：8389] - [当前时间：17:44:14]

[ok] - 386030742722
[尝试次数：1] - [剩余数量：8388] - [当前时间：17:44:14]

[ok] - 3

[ok] - 385119414239
[尝试次数：1] - [剩余数量：8270] - [当前时间：17:44:51]

[ok] - 383451039961
[尝试次数：1] - [剩余数量：8269] - [当前时间：17:44:51]

[ok] - 385119414239
[尝试次数：2] - [剩余数量：8268] - [当前时间：17:44:52]

[ok] - 385119414239
[尝试次数：4] - [剩余数量：8267] - [当前时间：17:44:52]

[ok] - 385119414239
[尝试次数：3] - [剩余数量：8266] - [当前时间：17:44:53]

[ok] - 385119414239
[尝试次数：1] - [剩余数量：8265] - [当前时间：17:44:53]

[ok] - 385119414239
[尝试次数：1] - [剩余数量：8264] - [当前时间：17:44:53]

[ok] - 385119414239
[尝试次数：1] - [剩余数量：8263] - [当前时间：17:44:53]

[ok] - 383451039961
[尝试次数：3] - [剩余数量：8262] - [当前时间：17:44:54]

[ok] - 385119414239
[尝试次数：1] - [剩余数量：8261] - [当前时间：17:44:54]

[ok] - 385119414239
[尝试次数：1] - [剩余数量：8260] - [当前时间：17:44:55]

[ok] - 385119414239
[尝试次数：1] - [剩余数量：8259] - [当前时间：17:44:55]

[ok] - 385119414239
[尝试次数：1] - [剩余数量：8258] - [当前时间：17:44:56]

[ok] - 385119414239
[尝试次数：1] - [剩余数量：8257] - [当前时间：17:44:56]

[ok] - 385119414239
[尝试次数：1] - [剩余数量：8256] - [当前时间：17:44:56]

[ok] - 385119414239
[尝试次数：1] - [剩余数量：8255] - [当前时间：17:44:56]

[ok] - 3

[ok] - 384983048803
[尝试次数：5] - [剩余数量：8136] - [当前时间：17:45:32]

[ok] - 385327910377
[尝试次数：3] - [剩余数量：8135] - [当前时间：17:45:32]

[ok] - 384983048803
[尝试次数：1] - [剩余数量：8134] - [当前时间：17:45:33]

[ok] - 384983048803
[尝试次数：1] - [剩余数量：8133] - [当前时间：17:45:33]

[ok] - 384983048803
[尝试次数：1] - [剩余数量：8132] - [当前时间：17:45:33]

[ok] - 384983048803
[尝试次数：1] - [剩余数量：8131] - [当前时间：17:45:33]

[ok] - 384983048803
[尝试次数：1] - [剩余数量：8130] - [当前时间：17:45:34]

[ok] - 384983048803
[尝试次数：2] - [剩余数量：8129] - [当前时间：17:45:35]

[ok] - 384983048803
[尝试次数：1] - [剩余数量：8128] - [当前时间：17:45:35]

[ok] - 384983048803
[尝试次数：3] - [剩余数量：8127] - [当前时间：17:45:35]

[ok] - 384983048803
[尝试次数：1] - [剩余数量：8126] - [当前时间：17:45:35]

[ok] - 384983048803
[尝试次数：1] - [剩余数量：8125] - [当前时间：17:45:35]

[ok] - 384983048803
[尝试次数：1] - [剩余数量：8124] - [当前时间：17:45:36]

[ok] - 384983048803
[尝试次数：5] - [剩余数量：8123] - [当前时间：17:45:36]

[ok] - 384983048803
[尝试次数：1] - [剩余数量：8122] - [当前时间：17:45:36]

[ok] - 384983048803
[尝试次数：3] - [剩余数量：8121] - [当前时间：17:45:36]

[ok] - 3

[ok] - 383416853734
[尝试次数：1] - [剩余数量：8003] - [当前时间：17:46:10]

[ok] - 383416853734
[尝试次数：3] - [剩余数量：8002] - [当前时间：17:46:11]

[ok] - 383416853734
[尝试次数：2] - [剩余数量：8001] - [当前时间：17:46:11]

[ok] - 383416853734
[尝试次数：7] - [剩余数量：8000] - [当前时间：17:46:11]

[ok] - 383416853734
[尝试次数：3] - [剩余数量：7999] - [当前时间：17:46:11]

[ok] - 383416853734
[尝试次数：1] - [剩余数量：7998] - [当前时间：17:46:11]

[ok] - 383416853734
[尝试次数：1] - [剩余数量：7997] - [当前时间：17:46:12]

[ok] - 383416853734
[尝试次数：2] - [剩余数量：7996] - [当前时间：17:46:12]

[ok] - 383416853734
[尝试次数：5] - [剩余数量：7995] - [当前时间：17:46:12]

[ok] - 383416853734
[尝试次数：1] - [剩余数量：7994] - [当前时间：17:46:13]

[ok] - 383416853734
[尝试次数：3] - [剩余数量：7993] - [当前时间：17:46:13]

[ok] - 383416853734
[尝试次数：5] - [剩余数量：7992] - [当前时间：17:46:14]

[ok] - 383416853734
[尝试次数：1] - [剩余数量：7991] - [当前时间：17:46:14]

[ok] - 383416853734
[尝试次数：1] - [剩余数量：7990] - [当前时间：17:46:14]

[ok] - 383416853734
[尝试次数：1] - [剩余数量：7989] - [当前时间：17:46:14]

[ok] - 383416853734
[尝试次数：4] - [剩余数量：7988] - [当前时间：17:46:14]

[ok] - 3